# 아이폰 가격 통계

### 필요한 라이브러리 로드

In [24]:
# 라이브러리 로드
from selenium import webdriver
from bs4 import BeautifulSoup
import time

### 네이버 쇼핑 상세검색 이용

In [25]:
# 검색어와 제외 검색어 리스트(영문 입력)
search_name_list = ["iphone_12_mini_64GB", "iphone_12_mini_128GB", "iphone_12_mini_256GB",
                    "iphone_12_64GB", "iphone_12_128GB", "iphone_12_256GB",
                    "iphone_12_pro_128GB", "iphone_12_pro_256GB", "iphone_12_pro_512GB",
                    "iphone_12_pro_max_128GB", "iphone_12_pro_max_256GB", "iphone_12_pro_max_512GB"]
search_except_name_list = ["128GB, 256GB, pro, max","64GB, 256GB, pro, max", "64GB, 128GB, pro, max",
                           "128GB, 256GB, mini, pro", "64GB, 256GB, mini, pro", "64GB, 128GB, mini, pro",
                           "256GB, 512GB, mini, max", "128GB, 512GB, mini, max", "128GB, 256GB, mini, max",
                           "256GB, 512GB, mini", "128GB, 512GB, mini","128GB, 256GB, mini"]
num = len(search_name_list)
print(len(search_name_list))
print(len(search_except_name_list))

12
12


### 아이폰 모델, 용량 별 가격 모아 파일로 출력하기

In [26]:
# 평균, 최고, 최소값 행 표시
price_index = ["평균가", "최고가", "최소가", "자료 개수"]
# 모든 상품 가격 리스트
all_price = []
# 모든 상품 가격과 행 정보 리스트
all_info = []

# 상품 별 가격 리스트화 하기
driver = webdriver.Chrome('chromedriver')
time.sleep(1)

for i in range(num):
    # webdriver 불러오기, '네이버 쇼핑' 접속
    url = "https://shopping.naver.com/home/p/index.naver"
    driver.get(url)
    time.sleep(2)
    
    # 네이버 쇼핑에 search_name_list의 요소들 하나씩 검색
    # 상세 검색으로 필요없는 단어 설정해서 검색하기 과정
    search_name = search_name_list[i]
    search_except_name = search_except_name_list[i]
    
    # 상세 검색 여는 버튼 xpath = '//*[@id="_detailSearchButton"]'
    # 상세 검색의 기본검색 창 xpath = '//*[@id="lb_srh_basis"]'
    # 제외하는 단어 검색창 xpath= '//*[@id="lb_srh_exclude"]'
    # 상세 검색 버튼 xpath = '//*[@id="_detailSearchDiv"]/div/div[2]/a[3]'
    # 정확히 검색창 xpath = '//*[@id="lb_srh_exact"]'
    to_detail_search_button = driver.find_element_by_xpath('//*[@id="_detailSearchButton"]')
    detail_search_basic_search = driver.find_element_by_xpath('//*[@id="lb_srh_basis"]')
    except_word = driver.find_element_by_xpath('//*[@id="lb_srh_exclude"]')
    detail_search_button = driver.find_element_by_xpath('//*[@id="_detailSearchDiv"]/div/div[2]/a[3]')
    exact_search = driver.find_element_by_xpath('//*[@id="lb_srh_exact"]')
    
    # 상세 검색창 클릭
    to_detail_search_button.click()
    time.sleep(1)
    # 상세 검색창에 검색어 입력
    detail_search_basic_search.send_keys(search_name)
    time.sleep(0.5)
    exact_search.send_keys(search_name)
    time.sleep(0.5)
    
    # 상세 검색에 제외 단어 입력하기
    except_word.send_keys(search_except_name)
    time.sleep(0.5)
    detail_search_button.click()
    time.sleep(2)
    
    # 페이지 스크롤 맨밑까지 내리기
    # 스크롤 높이 가져옴
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # 끝까지 스크롤 다운
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)

        # 스크롤 다운 후 스크롤 높이 다시 가져옴
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    
    # 상품의 가격 수집(BS)
    page = driver.page_source
    soup = BeautifulSoup(page, 'lxml')
    
    # 전체 상품 리스트 담기 태그 : 'ul', class_='list_basis'
    all_mer_list = soup.find('ul', class_='list_basis')
    
    # 가격표시 강조 부분 태그 : 'strong', class_='basicList_price__2r23_'
    basiclist_price = all_mer_list.find_all('strong', class_='basicList_price__2r23_')
    
    # 상품별 가격 모으기
    price_list = []
    for one in basiclist_price:
        tmp = one.find('span', class_='price_num__2WUXn').text
        price_list.append(tmp)
        
    # 가격을 숫자 형태로 변환
    int_price_list = []
    for i in list(price_list):
        tmp = i.replace("원", "")
        tmp = tmp.replace(",","")
        tmp = int(tmp)
        int_price_list.append(tmp)
        
    # 극단값 제거
    mod_price_list = []
    for i in int_price_list:
        if i >= 500000 and i <= 2000000:
            mod_price_list.append(i)
        
    # 평균, 최고, 최저가 출력
    mean = int(sum(mod_price_list) / len(mod_price_list)) # 정수값으로 출력
    maxium = max(mod_price_list)
    minium = min(mod_price_list)
    num_mod_price_list = len(mod_price_list) # 크롤링한 자료 개수
    # 확인
    print(f'{search_name}의 평균가격 :', mean,'원')
    print(f'{search_name}의 최고가격 :', maxium,'원')
    print(f'{search_name}의 최저가격 :', minium,'원')
    print(f"총 {num_mod_price_list}개")
    
    # 평균, 최고, 최저가 순서대로 리스트로 만들기
    mod_price = []
    mod_price.append(mean)
    mod_price.append(maxium)
    mod_price.append(minium)
    mod_price.append(num_mod_price_list)
       
    # 여러 상품의 가격을 하나로 모아 리스트 만들기
    all_price.append(mod_price)
    time.sleep(2)
    
# 여러 상품의 가격과 행 정보를 모아 파일로 만들기
import pandas as pd
dict_dat = {}
dict_dat["정보"] = price_index
for i in range(num):
    dict_dat[search_name_list[i]] = all_price[i]
dat = pd.DataFrame(dict_dat)
dat.to_csv("iphone_price_by_model.csv", index=False)
dat.to_excel("iphone_price_by_model.xlsx", index=False)

iphone_12_mini_64GB의 평균가격 : 866130 원
iphone_12_mini_64GB의 최고가격 : 1330000 원
iphone_12_mini_64GB의 최저가격 : 559000 원
총 37개
iphone_12_mini_128GB의 평균가격 : 1107784 원
iphone_12_mini_128GB의 최고가격 : 1530000 원
iphone_12_mini_128GB의 최저가격 : 529000 원
총 37개
iphone_12_mini_256GB의 평균가격 : 1300857 원
iphone_12_mini_256GB의 최고가격 : 1910000 원
iphone_12_mini_256GB의 최저가격 : 672000 원
총 38개
iphone_12_64GB의 평균가격 : 1163326 원
iphone_12_64GB의 최고가격 : 1420000 원
iphone_12_64GB의 최저가격 : 875990 원
총 40개
iphone_12_128GB의 평균가격 : 1290010 원
iphone_12_128GB의 최고가격 : 1740000 원
iphone_12_128GB의 최저가격 : 516300 원
총 39개
iphone_12_256GB의 평균가격 : 1416152 원
iphone_12_256GB의 최고가격 : 1950000 원
iphone_12_256GB의 최저가격 : 799900 원
총 39개
iphone_12_pro_128GB의 평균가격 : 1258209 원
iphone_12_pro_128GB의 최고가격 : 1470000 원
iphone_12_pro_128GB의 최저가격 : 571300 원
총 39개
iphone_12_pro_256GB의 평균가격 : 1468485 원
iphone_12_pro_256GB의 최고가격 : 1690000 원
iphone_12_pro_256GB의 최저가격 : 835300 원
총 38개
iphone_12_pro_512GB의 평균가격 : 1815493 원
iphone_12_pro_512GB의 최고가격 : 1990000 원
iphone

### csv 파일을 읽어 막대그래프 만들기(pandas, plotly 이용)

In [27]:
# pandas 로드
import pandas as pd

In [9]:
# csv 파일 읽어오기
ipp = pd.read_csv("iphone_price_by_model.csv")
ipp.head()

,정보,iphone_12_mini_64GB,iphone_12_mini_128GB,iphone_12_mini_256GB,iphone_12_64GB,iphone_12_128GB,iphone_12_256GB,iphone_12_pro_128GB,iphone_12_pro_256GB,iphone_12_pro_512GB,iphone_12_pro_max_128GB,iphone_12_pro_max_256GB,iphone_12_pro_max_512GB
0,평균가,866130,1107784,1300857,1163326,1290010,1416152,1258209,1468485,1815493,1082486,1199708,1467542
1,최고가,1330000,1530000,1910000,1420000,1740000,1950000,1470000,1690000,1990000,1474000,1606000,1870000
2,최소가,559000,529000,672000,875990,516300,799900,571300,835300,1409000,605760,798060,1090400
3,자료 개수,37,37,38,40,39,39,39,38,39,39,39,39


In [28]:
ipp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   정보                       4 non-null      object
 1   iphone_12_mini_64GB      4 non-null      int64 
 2   iphone_12_mini_128GB     4 non-null      int64 
 3   iphone_12_mini_256GB     4 non-null      int64 
 4   iphone_12_64GB           4 non-null      int64 
 5   iphone_12_128GB          4 non-null      int64 
 6   iphone_12_256GB          4 non-null      int64 
 7   iphone_12_pro_128GB      4 non-null      int64 
 8   iphone_12_pro_256GB      4 non-null      int64 
 9   iphone_12_pro_512GB      4 non-null      int64 
 10  iphone_12_pro_max_128GB  4 non-null      int64 
 11  iphone_12_pro_max_256GB  4 non-null      int64 
 12  iphone_12_pro_max_512GB  4 non-null      int64 
dtypes: int64(12), object(1)
memory usage: 544.0+ bytes


In [29]:
# plotly, cufflinks 로드
import plotly
import cufflinks as cf

In [30]:
# 오프라인 모드에서도 인터렉티브한 그래픽을 가능하도록 하기
# Enabling the offline mode for interactive plotting locally
from plotly.offline import download_plotlyjs,init_notebook_mode,plot,iplot
init_notebook_mode(connected=True)
cf.go_offline()

In [31]:
# plotly 이용 막대 그래프 그리기
ipp.iloc[0:3].iplot(kind='bar', x="정보")